In [1]:
import copy
import math
import random
import time

import numpy as np
import torch
import torch.optim as optim

from pfedgraph_gcosine.config import get_args
from pfedgraph_gcosine.utils import aggregation_by_graph, update_graph_matrix_neighbor, compute_acc, compute_local_test_accuracy
from prepare_data import get_dataloader
from attack import *
from model import coragcn, niidgcn
from graphdataset import graphdataset_read

In [2]:
from scipy.spatial.distance import cosine, cdist


In [6]:
dataset = 'pubmed'
beta = 0.5
distr = 'noniid'

In [4]:
datasets, traindata_cls_counts_npy, data_distributions, val, _ = graphdataset_read(dataset, 10, 'noniid', beta, 3)

config:  {'fedgraph_task': 'NC', 'dataset': 'pubmed', 'method': 'FedGCN', 'iid_beta': 0.5, 'distribution_type': 'average', 'n_trainer': 10, 'batch_size': -1, 'num_hops': 1, 'use_huggingface': False, 'num_nodes_to_remove': 100}
File already exists: ./data/pubmed/raw/ind.pubmed.x
File already exists: ./data/pubmed/raw/ind.pubmed.tx
File already exists: ./data/pubmed/raw/ind.pubmed.allx
File already exists: ./data/pubmed/raw/ind.pubmed.y
File already exists: ./data/pubmed/raw/ind.pubmed.ty
File already exists: ./data/pubmed/raw/ind.pubmed.ally
File already exists: ./data/pubmed/raw/ind.pubmed.graph
File already exists: ./data/pubmed/raw/ind.pubmed.test.index


In [5]:
traindata_cls_counts_npy

array([[8.020e+02, 1.471e+03, 0.000e+00],
       [3.000e+00, 5.500e+01, 7.320e+02],
       [4.260e+02, 5.590e+02, 1.985e+03],
       [9.700e+01, 4.690e+02, 4.290e+02],
       [4.870e+02, 3.400e+01, 2.530e+02],
       [3.000e+00, 2.001e+03, 2.530e+02],
       [4.000e+00, 1.000e+01, 2.626e+03],
       [3.320e+02, 3.540e+02, 3.300e+01],
       [1.021e+03, 1.177e+03, 3.300e+01],
       [1.060e+02, 1.000e+00, 2.270e+02]])

In [5]:
datasets[0].test_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [6]:
val.test_mask

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [8]:
import scipy
import numpy
import os
import pickle

In [9]:
dists = []
for dataset in datasets:
    dist = cdist(dataset.x, dataset.x)
    print(dist.shape)
    dists.append(dist)
D_partial = scipy.linalg.block_diag(*dists)

(2841, 2841)
(987, 987)
(3712, 3712)
(1243, 1243)
(967, 967)
(2505, 2505)
(3300, 3300)
(898, 898)
(2747, 2747)
(417, 417)


In [13]:
print(type(traindata_cls_counts_npy))

<class 'numpy.ndarray'>


In [10]:
import numpy
import os
import pickle

In [9]:
base = '/home/sattu/pFedGraph/data/'

In [11]:
name  = base + dataset + '/' + distr  + '/' + str(beta).replace('.', '_') 
os.makedirs(name, exist_ok=True)
numpy.save(name + '/D_partial.npy', D_partial)

In [12]:
with open(name + '/datasets.pkl', "wb") as f:
    pickle.dump(datasets,f)
numpy.save(name + '/traindata_cls_counts_npy.npy', traindata_cls_counts_npy)
numpy.save(name + '/data_distributions.npy', data_distributions)

In [13]:
torch.save(val, name + '/val.pt')

In [14]:
import numpy as np
datasets = np.load('/home/sattu/pFedGraph/data/pubmed/noniid/0_5/datasets.pkl', allow_pickle=True)
n = 0
l = [0]
for dataset in datasets:
    n += dataset.x.shape[0]
    l.append(n)
C = np.zeros((n, len(datasets)))
print(n)
for i in range(len(l) - 1):
    C[l[i]:l[i+1], i] = 1


19617


In [19]:
C

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [15]:
np.save("/home/sattu/pFedGraph/data/pubmed/noniid/0_5/C.npy", C)